# Tutorial: Extracting Information from Text Using OpenAI's API

This tutorial demonstrates how to use OpenAI's API to extract information from a text input. We will go through the steps of setting up the OpenAI client, making requests to the API, and processing the responses. The example involves summarizing a Danish news article and extracting structured information from it.

## Installation and Imports
First, we need to install the required library and import necessary modules.


In [2]:
!pip install openai -q

In [8]:
# Import required libraries
from openai import OpenAI
from google.colab import userdata
import json
from pydantic import BaseModel, Field

Setting Up the OpenAI Client

We will set up the OpenAI client using a custom API key and base URL.

In [9]:
# Setup OpenAI client with custom API key and base URL
TOGETHER_API_KEY = userdata.get('TOGETHER_API_KEY')

### Summarizing Text

We will call the language model to summarize a given Danish text into a single sentence.

In [11]:
# Create client
client = OpenAI(
    base_url="https://api.together.xyz/v1",
    api_key=TOGETHER_API_KEY
)

In [29]:
text = """
En nu 16-årig dreng fra Grenaa er blevet idømt otte års fængsel for et knivstikkeri i november sidste år på letbanestationen i Grenaa, hvor en 15-årig dreng mistede livet.
Et nævningeting ved Retten i Randers har kendt den 16-årige skyldig i drab efter at have stukket den 15-årige i brystkassen med en dolk.
Han er også kendt skyldig i forsøg på grov vold mod en anden dreng, der også var til stede.
De to unge havde i længere tid haft nogle uoverensstemmelser, og de mødtes en søndag aften sammen med andre unge på letbanestationen midt i Grenaa.
Her tog den 15-årige fat i den 16-årige og gav ham et knytnæveslag i ansigtet. Derefter tog den 16-årige sin dolk frem og stak ham.
Knivstikket ramte den 15-årige i hjertet, og han døde kort efter.
"""

In [30]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant extracting information from Danish text.",
        },
        {
            "role": "user",
            "content": "Summarize following in 1 sentence: " + text ,
        },
    ],
)

output = chat_completion.choices[0].message.content

In [ ]:
print(output)

## Creating a User Object and extracting structured info
We will define a schema for a user and call the API to create a user object based on this schema.

In [32]:
# Define the schema for the output.
class User(BaseModel):
    name: str = Field(description="user name")
    address: str = Field(description="address")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    response_format={"type": "json_object", "schema": User.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers in JSON.",
        },
        {
            "role": "user",
            "content": "Create a user named Alice, who lives in 42, Wonderland Avenue.",
        },
    ],
)

created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, indent=2))

## Extracting Case Details
We will define a schema for case details and extract relevant information from the given text.

In [34]:
class CaseDetails(BaseModel):
    incident_date: str = Field(description="Date of the incident")
    incident_location: str = Field(description="Location of the incident")
    incident_description: str = Field(description="Description of the incident")
    victim_age: int = Field(description="Age of the victim")
    victim_outcome: str = Field(description="Outcome for the victim")
    perpetrator_age: int = Field(description="Age of the perpetrator")
    perpetrator_sentence: str = Field(description="Sentence given to the perpetrator")
    perpetrator_charges: list[str] = Field(description="Charges against the perpetrator")
    trial_court: str = Field(description="Court where the trial was held")
    trial_verdict: str = Field(description="Verdict of the trial")

In [ ]:
# Call the LLM with the JSON schema
chat_completion = client.chat.completions.create(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    response_format={"type": "json_object", "schema": CaseDetails.model_json_schema()},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant that answers in JSON.",
        },
        {
            "role": "user",
            "content": "Extract case informatin form following.: " + text,
        },
    ],
)

created_user = json.loads(chat_completion.choices[0].message.content)
print(json.dumps(created_user, indent=2))